In [1]:
import os
import ast
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
tqdm.pandas()

In [2]:
df = pd.read_csv('../data/search/JDOviedoAr_sentiment.csv')
df["date"] = pd.to_datetime(df["created_at"]).dt.date
df

,entities,author_id,id,text,created_at,public_metrics,edit_history_tweet_ids,username,name,sentiment_llm,date
0,"{'annotations': [{'start': 265, 'end': 272, 'p...",177823502,1973030623792296212,El RIDICULO que hicieron estos señores de la u...,2025-09-30 14:22:12+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1973030623792296212'],mikeariza,Miguel Ariza Castro,NEGATIVO,2025-09-30
1,"{'mentions': [{'start': 0, 'end': 15, 'usernam...",3145461383,1973029119152787676,@lavozdelojusto @JDOviedoAr @ConTodaPorCol @Me...,2025-09-30 14:16:13+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1973029119152787676'],08Rosareda,Andrea 🦕 🌌💋🐳🐙,NEUTRO,2025-09-30
2,"{'annotations': [{'start': 76, 'end': 79, 'pro...",3145461383,1973028989192314940,@Palabrejo @JDOviedoAr @ConTodaPorCol @MejorJu...,2025-09-30 14:15:42+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1973028989192314940'],08Rosareda,Andrea 🦕 🌌💋🐳🐙,NEUTRO,2025-09-30
3,"{'annotations': [{'start': 63, 'end': 71, 'pro...",3145461383,1973028873748238623,@527ElduroJadin @JDOviedoAr @ConTodaPorCol @Me...,2025-09-30 14:15:15+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1973028873748238623'],08Rosareda,Andrea 🦕 🌌💋🐳🐙,NEUTRO,2025-09-30
4,"{'annotations': [{'start': 60, 'end': 64, 'pro...",3145461383,1973028724628238409,@sircamilopinto @JDOviedoAr @ConTodaPorCol @Me...,2025-09-30 14:14:39+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1973028724628238409'],08Rosareda,Andrea 🦕 🌌💋🐳🐙,NEUTRO,2025-09-30
...,...,...,...,...,...,...,...,...,...,...,...
2646,"{'mentions': [{'start': 0, 'end': 7, 'username...",1597223287,1970534072402550958,@Hora20 @JDOviedoAr @susanamuhamad @PaolaHolgu...,2025-09-23 17:01:48+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1970534072402550958'],umabdalaziz3,BENJAMÍN,NEUTRO,2025-09-23
2647,"{'mentions': [{'start': 0, 'end': 7, 'username...",1488698923,1970533639801786425,@Hora20 @JDOviedoAr @susanamuhamad @PaolaHolgu...,2025-09-23 17:00:05+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1970533639801786425'],maha_875,lina maría,NEUTRO,2025-09-23
2648,"{'mentions': [{'start': 0, 'end': 7, 'username...",1378620511,1970533637004443968,@Hora20 @JDOviedoAr @susanamuhamad @PaolaHolgu...,2025-09-23 17:00:04+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1970533637004443968'],Pepiinno,ricardo,NEUTRO,2025-09-23
2649,"{'mentions': [{'start': 111, 'end': 126, 'user...",1638556330976747520,1970522817612652670,Carta Solicitud Reconciderar Cupos Escolares E...,2025-09-23 16:17:04+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1970522817612652670'],JeissonRayoR,Jeisson S. Rayo Rodríguez,NEUTRO,2025-09-23


In [3]:
conversation_ids = []
with open('../data/search/tweets_ids_video_oviedo.txt', 'r') as f:
    for line in f:
        if line.strip().isnumeric():
            conversation_ids.append(int(line.strip()))

In [4]:
conversation_ids[:5]

[1970940413612806652,
 1970947626427462119,
 1970918993943986611,
 1970914160080007281,
 1970953034525180123]

In [5]:
len(conversation_ids)

228

In [6]:
def is_related(row):
    return row['id'] in conversation_ids or any(p in row['text'].lower() for p in ['consejo', 'concejo', 'concejal'])

df['related'] = df.apply(lambda row: is_related(row), axis=1)
df.head()

,entities,author_id,id,text,created_at,public_metrics,edit_history_tweet_ids,username,name,sentiment_llm,date,related
0,"{'annotations': [{'start': 265, 'end': 272, 'p...",177823502,1973030623792296212,El RIDICULO que hicieron estos señores de la u...,2025-09-30 14:22:12+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1973030623792296212'],mikeariza,Miguel Ariza Castro,NEGATIVO,2025-09-30,False
1,"{'mentions': [{'start': 0, 'end': 15, 'usernam...",3145461383,1973029119152787676,@lavozdelojusto @JDOviedoAr @ConTodaPorCol @Me...,2025-09-30 14:16:13+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1973029119152787676'],08Rosareda,Andrea 🦕 🌌💋🐳🐙,NEUTRO,2025-09-30,False
2,"{'annotations': [{'start': 76, 'end': 79, 'pro...",3145461383,1973028989192314940,@Palabrejo @JDOviedoAr @ConTodaPorCol @MejorJu...,2025-09-30 14:15:42+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1973028989192314940'],08Rosareda,Andrea 🦕 🌌💋🐳🐙,NEUTRO,2025-09-30,False
3,"{'annotations': [{'start': 63, 'end': 71, 'pro...",3145461383,1973028873748238623,@527ElduroJadin @JDOviedoAr @ConTodaPorCol @Me...,2025-09-30 14:15:15+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1973028873748238623'],08Rosareda,Andrea 🦕 🌌💋🐳🐙,NEUTRO,2025-09-30,False
4,"{'annotations': [{'start': 60, 'end': 64, 'pro...",3145461383,1973028724628238409,@sircamilopinto @JDOviedoAr @ConTodaPorCol @Me...,2025-09-30 14:14:39+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1973028724628238409'],08Rosareda,Andrea 🦕 🌌💋🐳🐙,NEUTRO,2025-09-30,False


In [7]:
df['related'].value_counts()

related
False    2386
True      265
Name: count, dtype: int64

In [8]:
df_filter = df[df['related']]
df_filter

,entities,author_id,id,text,created_at,public_metrics,edit_history_tweet_ids,username,name,sentiment_llm,date,related
24,"{'annotations': [{'start': 217, 'end': 225, 'p...",741473318751109120,1972947446147039635,@JDOviedoAr @petrogustavo @ConTodaPorCol @Maur...,2025-09-30 08:51:41+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1972947446147039635'],JohaYpunto,JohaYPunto,NEGATIVO,2025-09-30,True
36,"{'mentions': [{'start': 0, 'end': 11, 'usernam...",779521198354997248,1972841162978525248,@JDOviedoAr Increíble que gente así sea candid...,2025-09-30 01:49:21+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1972841162978525248'],Don_abelardo,Abelardo Jaramillo,NEGATIVO,2025-09-30,True
40,"{'annotations': [{'start': 31, 'end': 37, 'pro...",862672071977054210,1972838272763384004,@JDOviedoAr Este señor es otro Rodolfo que feo...,2025-09-30 01:37:52+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1972838272763384004'],IslenBello1,𝓘𝓼𝓵𝓮𝓷 𝓮𝓵 𝔃𝓾𝓻𝓭𝓸,NEGATIVO,2025-09-30,True
41,"{'mentions': [{'start': 0, 'end': 11, 'usernam...",1797785276640579584,1972833141346927025,"@JDOviedoAr El corrupto disfrazado de ""buena g...",2025-09-30 01:17:29+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1972833141346927025'],bedoivanchini,Iván Bedoya,NEGATIVO,2025-09-30,True
47,"{'mentions': [{'start': 0, 'end': 11, 'usernam...",1950185162815524865,1972820641046057362,@JDOviedoAr ESTÁ POBRE LOCA NI HABKAR SABE …. ...,2025-09-30 00:27:48+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1972820641046057362'],BBernardo9904,Bernardo Rodriguez B.,NEGATIVO,2025-09-30,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2627,"{'mentions': [{'start': 92, 'end': 103, 'usern...",69028477,1970602163836510280,Qué gran noticia para Colombia! El Consejo de ...,2025-09-23 21:32:22+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1970602163836510280'],ERWINAGAMEZ,Erwin Agámez,POSITIVO,2025-09-23,True
2630,"{'mentions': [{'start': 77, 'end': 88, 'userna...",717440254303711232,1970594534305550620,"#Política | A través de este comunicado, el pr...",2025-09-23 21:02:03+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1970594534305550620'],DoomoEditorial,Doomo Editorial,NEUTRO,2025-09-23,True
2632,"{'mentions': [{'start': 21, 'end': 37, 'userna...",134524583,1970590781602717988,📢Grandes noticias 🗞️ @consejodeestado Mantuvo ...,2025-09-23 20:47:08+00:00,"{'retweet_count': 2, 'reply_count': 1, 'like_c...","['1970581221077066109', '1970590781602717988']",hollmanibanezp,Hollman Ibáñez Parra,POSITIVO,2025-09-23,True
2633,"{'mentions': [{'start': 16, 'end': 27, 'userna...",59858473,1970590599565709632,El precandidato @JDOviedoAr confirmó que el @c...,2025-09-23 20:46:25+00:00,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1970590599565709632'],Marovaan,Mauricio Vanegas,NEUTRO,2025-09-23,True


In [15]:
df_filter['sentiment_llm'].value_counts(normalize=True).to_frame('Count')

,Count
sentiment_llm,
NEGATIVO,0.615094
NEUTRO,0.249057
POSITIVO,0.135849


In [10]:
df_filter.iloc[0]['created_at']

'2025-09-30 08:51:41+00:00'

In [11]:
min(df_filter['date']), max(df_filter['date'])

(datetime.date(2025, 9, 23), datetime.date(2025, 9, 30))

In [12]:
df_filter.groupby('date').size().to_frame('Count')

,Count
date,
2025-09-23,8
2025-09-24,114
2025-09-25,49
2025-09-26,32
2025-09-27,2
2025-09-28,31
2025-09-29,22
2025-09-30,7


In [13]:
df_summ = pd.crosstab(df_filter['date'], df_filter['sentiment_llm']).reset_index()
df_summ

sentiment_llm,date,NEGATIVO,NEUTRO,POSITIVO
0,2025-09-23,0,3,5
1,2025-09-24,61,29,24
2,2025-09-25,31,13,5
3,2025-09-26,21,11,0
4,2025-09-27,2,0,0
5,2025-09-28,28,3,0
6,2025-09-29,15,6,1
7,2025-09-30,5,1,1
